# pyWECcast - v0.2.2
## WEC Powerseries Generator

This version release represents a refactoring to reduce dependence on specific file configurations for powerseries generation.

This is also the first implementation of numba accelerated functions with parallel functionality included


In [1]:
# General import directly from the local pyWECcast directory

import sys
sys.path.insert(1, '../')

import pyWECcast as wc

In [2]:
baseDir = f'/mnt/c/Users/abharath/Documents/Projects/Grid_Value'
dataDir = f'{baseDir}/data'
buoyDB = f'{dataDir}/buoy_downloads.h5'
resultDB = f'{dataDir}/testing_buoy_powerseries.h5'

In [3]:
# First thing is first, we must compile the sea-state data we wish to use to base our WEC 
# power series on

# For a basic reconstruction we need waveheight and period timeseries, but for a directional
# reconstruction we will require directional data as well

# This example is based on the swell wave data contained within the NOAA Buoy data

from h5py import File

with File(buoyDB,'r') as buoys:
    buoy = buoys['46041']
    SwH, SwP, SwD = buoy['SwH'][:], buoy['SwP'][:], buoy['SwD'][:]
    time_index = buoy['time_index'][:]

print(SwH, SwP, SwD, time_index)

[3.4 3.  3.2 ... 2.  1.6 1.7] [11.4 10.8 11.4 ... 10.8 10.8 11.4] [b'WSW' b'SW' b'WSW' ... b'NW' b'WNW' b'NW'] [b'2020-01-29 00:00:00' b'2020-01-29 01:00:00' b'2020-01-29 02:00:00' ...
 b'2020-03-13 19:00:00' b'2020-03-13 20:00:00' b'2020-03-13 21:00:00']


In [4]:
# we also include convenience loading functions for NOAA buoy data files (based on those 
# constructed using the pyWECcast download utility)

Hs = 'SwH'
Tp = 'SwP'
Dir = 'SwD'

time_index, SwH, SwP, SwD = wc.extract_NOAA_buoy(buoyDB,'46041',
                                                   Hs, Tp, Dir)

print(SwH, SwP, SwD, time_index)

[3.4 3.  3.2 ... 2.  1.6 1.7] [11.4 10.8 11.4 ... 10.8 10.8 11.4] [b'WSW' b'SW' b'WSW' ... b'NW' b'WNW' b'NW'] [b'2020-01-29 00:00:00' b'2020-01-29 01:00:00' b'2020-01-29 02:00:00' ...
 b'2020-03-13 19:00:00' b'2020-03-13 20:00:00' b'2020-03-13 21:00:00']


In [5]:
# for the particular NOAA buoy data files we have convenience functions in wc.buoy to convert to
# a numerical format.

timestamps = wc.ufuncs.binary_timestamp_convert(time_index)

sea_direction = wc.ufuncs.binary_direction_convert(SwD)


../pyWECcast/ufuncs/utility_functions.py:77: UserWarning: Directional ndarray contains NaNs - default value inserted 0.0
  warnings.warn(warn,UserWarning)


In [6]:
cut = 5
timestamps, SwH, SwP = timestamps[:cut], SwH[:cut], SwP[:cut]

In [7]:
# With a WECSim powerseries matrix, we can link the sea-state data to the WEC simulations 
# that are available. The closest match can be determined using a nearest neighbor method on 
# separate Hs, Tp and direction values or combinations of the groups

wecSim = f'{dataDir}/WECSim_dataset_RM3_scale_0-1873.hdf5'

wecSimSeas, measured = wc.link_sea_states(wecSim,SwH,SwP)

# we can also compare WECSim sea states and those measure to analyze the variations between them

#print([i for i in zip(wecSimSeas, measured)])

In [8]:
# With the sea-states in the dataset linked to what is available in the WECSim simulation db
# We can calculate the FFTs to obtian the reconstruction coefficients

fftFname = f'{dataDir}/testing_fft_db.h5'

# if inMemory:True, calculate_fft_matrix() will return coefficients as a dict
fft_matrix = wc.calculate_fft_matrix(wecSim,wecSimSeas[:,0],wecSimSeas[:,1],
                                  fftFname=fftFname,inMemory=True)

In [9]:
# With the coefficients and frequencies we can not reconstruct the powerseries
freq = '60S'

wc.construct_powerseries(timestamps,freq,wecSimSeas[:,0],wecSimSeas[:,1],
                         nWECs=5,fft_matrix=fft_matrix)

100%|██████████| 4/4 [00:00<00:00,  4.40it/s]


In [10]:
%matplotlib widget

import numpy as np

import matplotlib.pyplot as plt

for i in range(results.shape[-1]):
    plt.plot(time,results[:,i])
    
plt.plot(time,np.mean(results,axis=1),color='k')


NameError: name 'results' is not defined

In [ ]:
freq = '10S'
for i in range(50):
    results, time = wc.construct_powerseries(timestamps,freq,wecSimSeas[:,0],wecSimSeas[:,1],
                         nWECs=i+1,fft_matrix=fft_matrix)
    print(np.mean(np.mean(results,axis=1)),np.std(np.mean(results,axis=1)),i)

In [12]:
from h5py import File

a = File('tempRecon.h5','r')
print(a.keys())
print(a['reconstruction'][:])

<KeysViewHDF5 ['reconstruction', 'time']>
[[11655.35957044  1074.61958209   592.58331863   154.56889638
   2037.64618291]
 [  928.3136516    761.50081977  2642.7262277     24.59386278
    962.45552678]
 [  298.07404721   152.28431256 15687.24391904    86.11575939
    824.58466618]
 ...
 [  684.36022389  1288.30061092  1422.28856009 13389.35270386
    625.04066194]
 [  601.30570289  2377.43134424   529.63061998  1422.67480722
  18869.33226555]
 [  103.34575109   270.47090881  1098.87514582   213.27497224
    464.02479961]]
